In [1]:
import os
from os.path import isfile, join
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', '^GSPC']
tiingo_api_key = os.environ['TIINGO_API_KEY']
iex_api_key = os.environ['IEX_API_KEY']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2010-01-01'
end_date = '2016-12-31'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
#panel_data = pdr.get_data_tiingo('GOOG', api_key=tiingo_api_key)
#panel_data = pdr.data.DataReader('GOOG', 'iex', start_date, api_key=iex_api_key)

In [3]:
# File containing all tickers listed by NASDAQ-100
tickers_file = 'nasdaq_100_tickers.txt'
data_dir = 'data/'
raw_data_dir = data_dir + 'raw/'

if not os.path.exists(raw_data_dir):
        os.makedirs(raw_data_dir)

In [4]:
# Read all the tickers to be used in the data
ndxt_tickers = []
with open(data_dir+tickers_file) as f:
    for ticker in f:
        ndxt_tickers.append(ticker.replace('\n', ''))

In [5]:
# Code for downloading data and saving it, use only when necessary
'''
raw_stock_data_tiingo = []
raw_stock_data_iex = []
error_tickers = []

for ticker in ndxt_tickers:
    try:
        raw_stock_data_tiingo.append(pdr.get_data_tiingo(ticker, api_key= tiingo_api_key))
    except:
        error_tickers.append(ticker)

# Save each stock data in a CSV file
for t in raw_stock_data_tiingo:
    t.to_csv(raw_data_dir + t.index.values[0][0] + '.csv')
'''

"\nraw_stock_data_tiingo = []\nraw_stock_data_iex = []\nerror_tickers = []\n\nfor ticker in ndxt_tickers:\n    try:\n        raw_stock_data_tiingo.append(pdr.get_data_tiingo(ticker, api_key= tiingo_api_key))\n    except:\n        error_tickers.append(ticker)\n\n# Save each stock data in a CSV file\nfor t in raw_stock_data_tiingo:\n    t.to_csv(raw_data_dir + t.index.values[0][0] + '.csv')\n"

In [6]:
# Read downloaded data from files
raw_stock_data = []
raw_index_data_filename = 'Nasdaq 100 Historical Data.csv'
raw_stock_data_filenames = [f for f in os.listdir(raw_data_dir) if isfile(join(raw_data_dir, f))]
raw_stock_data_filenames.remove(raw_index_data_filename)
raw_index_df = pd.read_csv(raw_data_dir + raw_index_data_filename)

for filename in raw_stock_data_filenames:
    raw_stock_data.append(pd.read_csv(raw_data_dir + filename))

In [7]:
# Check for incomplete(dates) stocks and remove them
stock_record_count = len(raw_stock_data[0])
for i, t in enumerate(raw_stock_data):
    if len(t) != stock_record_count: 
        raw_stock_data.pop(i)

In [8]:
# Check for missing values
for i, t in enumerate(raw_stock_data):
    if t.isnull().any().any(): print('Missing data.')

In [9]:
# Check that all stock data have the same dates, not the most optimal way to calculate it yet since its O(n^2)
equal = True
for i in range(len(raw_stock_data)):
    for j in range(len(raw_stock_data)):
        if not (raw_stock_data[i]['date'] == raw_stock_data[j]['date']).all(): equal = False
print('Data has equal dates in all rows: ' + str(equal))

Data has equal dates in all rows: True


In [10]:
# Reformat date in stocks dataframes 
for data in raw_stock_data:
    data['date'] = data['date'].map(lambda x: x.split()[0])
    
# Reformat date in index dataframe
mon_to_num = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

raw_index_df['Date'] = raw_index_df['Date'].map(lambda x: x.split()[2]+'-'+ mon_to_num[x.split()[0]] +'-'+x.split()[1].strip(','))

In [11]:
# Rename index columns
raw_index_df.columns = ['date', 'close', 'open', 'high', 'low', 'volume', 'change']

In [12]:
# Remove commas from numbers
raw_index_df['close'] = raw_index_df['close'].map(lambda x: float(x.replace(',', '')))
raw_index_df['open'] = raw_index_df['open'].map(lambda x: float(x.replace(',', '')))
raw_index_df['high'] = raw_index_df['high'].map(lambda x: float(x.replace(',', '')))
raw_index_df['low'] = raw_index_df['low'].map(lambda x: float(x.replace(',', '')))

# Reformat volume values in raw_index_df
raw_index_df['volume'] = raw_index_df['volume'].map(lambda x: int(float(x.strip('M'))*1000000))

# Reformat change values and normalize them
raw_index_df['change'] = raw_index_df['change'].map(lambda x: float(x.strip('%')))

In [13]:
# Assign symbol and date as index identifiers for every stock record
for data in raw_stock_data:
    data.set_index(['symbol', 'date'], inplace=True, drop=True)
    
# Assign date as index identifier for index records as well
raw_index_df.set_index(['date'], inplace=True, drop=True)

In [14]:
# Concatenate all stock datadrames into one
raw_stock_df = pd.concat(raw_stock_data)

In [15]:
# Remove unnecessary information
raw_stock_df.drop(columns=['divCash', 'splitFactor'], inplace=True)

In [16]:
# Reverse stock records
raw_stock_df = raw_stock_df.iloc[::-1]

# Trim index or stock data so they both contain the same date ranges. 
index_record_count = len(raw_index_df)
if stock_record_count > index_record_count:
    for symbol, df in raw_stock_df.groupby(level=0):
        df = df.iloc[:index_record_count,:]
else:
    raw_index_df = raw_index_df.iloc[:stock_record_count, :]
    
# Assert the lengths of index and individual stocks dataframes
assert len(raw_index_df) == len(raw_stock_df.groupby(level=1))

# Assert their inicial and end dates
assert raw_index_df.index[0] == raw_stock_df.first_valid_index()[1]
assert raw_index_df.index[-1] == raw_stock_df.last_valid_index()[1]

In [17]:
# DataFrames have been processed and not considered raw anymore
stocks_df = pd.DataFrame(raw_stock_df)
index_df = pd.DataFrame(raw_index_df)

In [18]:
def change(stock_df, period = 1):
    change = []
    for i in range(len(stock_df)):
        try:
            today = stock_df.iloc[i]['close']
            previous = stock_df.iloc[i+period]['close']
            change.append(100*(today-previous)/previous)
        except:
            change.append(None)
    stock_df.insert(loc=0, column='change', value=change)

In [19]:
def PMO(stock_df, period = 10):
    pmo = []
    for i in range(len(stock_df)):
        try:
            today = stock_df.iloc[i]['close']
            previous = stock_df.iloc[i+period]['close']
            pmo.append(today - previous)
        except:
            pmo.append(None)
    stock_df.insert(loc=0, column='PMO', value=pmo)

In [20]:
def RSI(stock_df, period = 10):
    rsi = []
    for i in range(len(stock_df)):
        try:
            rsi_value = 0
            pos, p = 0, 0
            neg, n = 0, 0
            
            for j in range(period):
                change = stock_df.iloc[i+j]['change']
                if change > 0: 
                    pos += change
                    p += 1
                elif change < 0: 
                    neg += abs(change)
                    n += 1
            
            if p: pos /= p
            if n: neg /= n
                
            if not pos:
                rsi_value = 0
            elif not neg:
                rsi_value = 100
            else:
                rsi_value = 100 - (100/(1+(pos/neg)))
            rsi.append(rsi_value)
        except:
            rsi.append(None)
    stock_df.insert(loc=0, column='RSI', value=rsi)

In [21]:
RSI(index_df)
PMO(index_df)
index_df

,RSI,close,open,high,low,volume,change
date,,,,,,,
2019-09-30,45.945946,7749.45,7705.55,7761.84,7693.73,144520000,0.88
2019-09-27,44.932845,7681.58,7786.03,7790.80,7626.82,170020000,-1.16
2019-09-26,50.136240,7771.99,7794.28,7798.12,7718.69,146930000,-0.40
2019-09-25,45.981773,7803.54,7706.50,7822.43,7647.63,154550000,1.21
2019-09-24,42.656113,7710.04,7855.97,7873.25,7684.80,205300000,-1.39
2019-09-23,53.591160,7818.61,7815.23,7842.98,7790.29,138570000,-0.06
2019-09-20,51.322751,7823.55,7901.46,7920.53,7794.27,425830000,-0.99
2019-09-19,62.742561,7901.79,7904.40,7950.21,7888.31,136330000,0.17
2019-09-18,75.909661,7888.56,7877.58,7891.17,7791.58,148230000,-0.00
